# IndoML 2023 Tuturial: Part 2
## At Sesame street

### Plan:

1. We will use Huggingface transformers for finetuning pretrained models.
2. This will allow us to easily swap out models to find the most suitable one.
    * This is achieved using the `AutoModel*` and `AutoTokenizer` classes.
3. It also comes with various new libraries integrated for efficient fine-tuning, 
multi-gpu or distributed training, and more.

### Methods that we will try:

1. DistilBERT
2. mBERT/distilmBERT (multilingual)
3. XLM-RoBERTa (multilingual)
4. DeBERTa (multilingual)

### Efficient Training

1. Linear Probing
2. PEFT
3. Distilled Models

### Libraries to explore

1. 🤗 Transfomers > Trainer
2. 🤗 Evaluate
2. wandb

## Load `dataset`

In [1]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tqdm.pandas()

dataset = load_dataset("AmazonScience/massive")


/home/bsantra/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-25 07:49:11.040538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 07:49:11.751707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load `AutoTokenizer`

In [2]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["utt"], truncation=True)



## Preprocessing

### Using the Trainer class

We will be using the HF Trainer class to handle a lot of the boilerplate tasks. This Trainer class expects certain format for the dataset.

1. Tokenize the dataset
2. A "labels" column should exits

In [3]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
tokenized_datasets['train'][0].keys()

dict_keys(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'input_ids', 'attention_mask'])

In [5]:
# Remove columns that are not needed
to_remove = list(tokenized_datasets['train'][0].keys())

# Don't remove these columns
to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('intent')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)
tokenized_datasets = tokenized_datasets.rename_column('intent', 'labels')


In [6]:
tokenized_datasets['train'].features

{'labels': ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_remove', 'transpor

## Dataloaders [Optional]

- Instead of going through individual samples in the dataset, we would like to 
batches of samples to train our model. 
- Dataloader
    1. Creates an "iterator" over the dataset, which returns a batch of samples every turn
    2. Handles shuffling
    3. Collates samples into batches, by padding the input sequences to the maximum length in the batch

In [7]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [8]:
# Check the shape of the tensors
# for batch in tqdm(train_dataloader):
#     break
# {k: v.shape for k, v in batch.items()}

```python
{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 25]),
 'attention_mask': torch.Size([8, 25])}
```

## Load model using an `AutoModel`

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=tokenized_datasets['train'].features['labels'].num_classes)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

In [10]:
# # Let's check the output of a single forward pass
# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

## Setup Evaluation Metric

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
# We need to define a compute_metric function that is supported by the Trainer output
# It basically converts the logits to predictions and then calls the metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Setup Trainer and Finetuning

1. When testing your code for the first time, it is a better idea to use a smaller dataset,
so that you can quickly iterate over your code.
2. We have done the same here by using 128 + 32 samples from the dataset, for training and validation respectively.

In [13]:
train_data_subset = tokenized_datasets["train"] #.select(range(4096))
eval_data_subset = tokenized_datasets["validation"] #.select(range(2096))

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    # warmup_steps=500,
    # weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none" # set "wandb" here to log to wandb
)

**Wandb Logging**

When training ML models, keeping track of various hyperparameters, the training and validation losses, and evaluation metrics is very important.
`wandb` can help you here.

To install wandb, run following commands in your terminal:

```bash
pip install wandb
# Then login to your wandb account
wandb login
```

Once, that is done, in TrainingArguments, set the `report_to` argument to `wandb`.

In [15]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    # train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["validation"],
    train_dataset=train_data_subset, # Using the smaller subsets
    eval_dataset=eval_data_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [16]:
stats = trainer.train()

/home/bsantra/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 587214
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 36702
  Number of trainable parameters = 66999612


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.479600,1.398437,0.636893
2,0.994500,1.289952,0.672020


***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-18351
Configuration saved in ./results/checkpoint-18351/config.json
Model weights saved in ./results/checkpoint-18351/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-18351/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-18351/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-36702
Configuration saved in ./results/checkpoint-36702/config.json
Model weights saved in ./results/checkpoint-36702/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-36702/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-36702/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-36702 (score: 0.6720195210400933).


In [17]:
# Print Stats
from pprint import pprint
pprint(trainer.evaluate())

***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32


{'epoch': 2.0,
 'eval_accuracy': 0.6720195210400933,
 'eval_loss': 1.2899516820907593,
 'eval_runtime': 31.7472,
 'eval_samples_per_second': 3265.893,
 'eval_steps_per_second': 102.088}
